In [1]:
import requests
import json
import pandas as pd

In [2]:
def round_down(n, places=2):
    return int(n * (10 ** places)) / 10 ** places

def us_to_decimal(odds):
    if odds > 1:
        ret = (odds/100) + 1
    else:
        ret = (-100/odds) + 1
        
    return round_down(ret, 2)
    
def decimal_to_us(odds):
    if odds > 2:
        return (100*(odds-1))
    else:
        return (-100/(odds-1))

def dec_prob(odds):
    return 1/odds
    

def get_home_away(event):
    home = ""
    away = ""

    for team in event['teams']:
        if team['is_home'] == True:
            home = team['name'].replace("\u00e9", "e")
        else:
            away = team['name'].replace("\u00e9", "e")
            
    return [home, away]

def process_odds(odds_lst, COMISSION = 0.03):
    com_each = len(odds_lst) / COMISSION
    new_lst = []
    
    for odds in odds_lst:
        odds = us_to_decimal(odds)
        odds = odds * (1-COMISSION)
        odds = round_down(odds, 2)
        new_lst.append(odds)
        
    return new_lst

In [3]:
headers = {
    'x-rapidapi-host': "therundown-therundown-v1.p.rapidapi.com",
    'x-rapidapi-key': "3036057f0emsh5177d1b9d9e1ff4p1e849djsn33b60cd42561"
    }

In [4]:
url = "https://therundown-therundown-v1.p.rapidapi.com"

In [5]:
res = requests.get(f'{url}/sports', headers=headers)

In [6]:
sports = json.loads(res.text)['sports']

In [7]:
sports = pd.DataFrame(sports)

In [8]:
len(sports)

15

In [9]:
sports['Sport'] = ['American Football', 'American Football', "Baseball", "Basketball", "Basketball", "Ice Hockey", "MMA", "Basketball", "Soccer", "Soccer", "Soccer", "Soccer", "Soccer", "Soccer", "Soccer"]

In [10]:
dates = pd.date_range("2022-03-10", "2022-06-10")

In [11]:
full = {}

In [12]:
for idx, row in sports.iterrows():

    print(row['Sport'])
    if row['Sport'] not in full:
        full[row['Sport']] = {}

    full[row['Sport']][row['sport_name']] = {}
    curr = []

    for d in dates:


        da = str(d.date())
        res = requests.get(f'{url}/sports/{row["sport_id"]}/events/{da}', headers=headers)
        events = json.loads(res.text)['events']
        for event in events:
            
            try:
                curr_event = {}
                curr_event['timestamp'] = event['event_date']
                curr_event['id'] = event['event_id']
                
                
                
                curr_event['match'] = get_home_away(event)
                
                
                if 'lines' in event:

                    for line in event['lines']:
                        ml_lines = event['lines'][line]['moneyline']

                        if ml_lines['moneyline_away'] > 1 or ml_lines['moneyline_away'] < -1:
                            odds = {}
                            odds_lst = []
                            
                            odds_lst.append(ml_lines['moneyline_home'])
                            odds_lst.append(ml_lines['moneyline_away'])
                            
                            if row['Sport'] == 'Soccer':
                                try:
                                    odds_lst.append(ml_lines['moneyline_draw'])
                                except:
                                    pass
                            
                            odds_lst = process_odds(odds_lst)
                            
                            
                            odds["1"] = odds_lst[0]
                            odds["2"] = odds_lst[1]
                            
                            if len(odds_lst) == 3:
                                odds["X"] = odds_lst[2]
                                
                            curr_event["outcomes"] = odds
                            curr.append(curr_event)
                            break
            except Exception as e:
                pass
    
    full[row['Sport']][row['sport_name']] = curr

American Football
American Football
Baseball
Basketball
Basketball
Ice Hockey
MMA
Basketball
Soccer
Soccer
Soccer
Soccer
Soccer
Soccer
Soccer


In [14]:
json.dump(full, open('../odds.json', 'w'))